# ETL Process 

In [90]:
#Importar Librerias
import pandas as pd
import re
import numpy as np
import os

In [91]:
#Carga CSV
df_movies = pd.read_csv('dataset/movies.csv', encoding='utf-8')
df_credits = pd.read_csv('dataset/credits.csv', encoding='utf-8')

/tmp/ipykernel_6128/4659526.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('dataset/movies.csv', encoding='utf-8')


In [92]:
df_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## Transformaciones

### Desanidar y eliminar campos compuestos 

In [93]:
#Funcion para extraer el valor de la clave 'name'
def extract_values(value):
    if isinstance(value, str):
        values = re.findall(r"'name': '(.*?)'", value)
        return ', '.join(values)
    else:
        return ''

#Crear de nuevos campos o modificar de campos existentes     
df_movies['collection'] = df_movies['belongs_to_collection'].apply(extract_values)
df_movies['genres'] = df_movies['genres'].apply(extract_values)
df_movies['companies'] = df_movies['production_companies'].apply(extract_values)
df_movies['country'] = df_movies['production_countries'].apply(extract_values)
df_movies['language'] = df_movies['spoken_languages'].apply(extract_values)

#Eliminacion de campos compuestos no modificados
df_movies = df_movies.drop(columns=['belongs_to_collection','production_companies',
                                    'production_countries','spoken_languages'])

#Eliminacion de campos innecesarios
df_movies = df_movies.drop(columns=['adult','homepage','imdb_id','original_title',
                                    'poster_path','video'])

### Creacion de campo con el retorno de inversion

In [94]:
#Convertir valores nulos del campo revenue a 0
df_movies['revenue'] = df_movies['revenue'].fillna(0)

#Convertir los valores del campo budget a Float64
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')
#Convertir valores nulos del campos budget a 0
df_movies['budget'] = df_movies['budget'].fillna(0)

#Creacion del campo return si los campos no son 0
df_movies['return'] = np.where((df_movies['revenue'] == 0) | (df_movies['budget'] == 0), 0,
                                df_movies['revenue'] / df_movies['budget'])

### Creacion de campo year y elminacion de filas con valores nulos del campo release_date 

In [95]:
#Convertir campo release_date a datetime 
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d', errors='coerce')
# Elimnacion de filas con valores nulos en en campo datetime
df_movies = df_movies.dropna(subset=['release_date'])

#Crear campo year
df_movies['year'] = df_movies['release_date'].dt.year

In [96]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             45376 non-null  float64       
 1   genres             45376 non-null  object        
 2   id                 45376 non-null  object        
 3   original_language  45365 non-null  object        
 4   overview           44435 non-null  object        
 5   popularity         45376 non-null  object        
 6   release_date       45376 non-null  datetime64[ns]
 7   revenue            45376 non-null  float64       
 8   runtime            45130 non-null  float64       
 9   status             45296 non-null  object        
 10  tagline            20398 non-null  object        
 11  title              45376 non-null  object        
 12  vote_average       45376 non-null  float64       
 13  vote_count         45376 non-null  float64       
 14  collection 

## Funciones

### def peliculas_idioma(idioma)
- Se ingresa un idioma (literal) y se retorna la cantidad de peliculas en ese idioma.

In [97]:
#Convertir campo original_language a string minuscula
df_movies['original_language'] = df_movies['original_language'].astype(str).str.lower()

In [98]:
#Funcion ingreso de idioma y retorno la cantidad de peliculas
def peliculas_idioma(idioma):
    idioma = idioma.lower()
    if idioma in df_movies['original_language'].values:
        df_peliculas_filtradas = df_movies[df_movies['original_language'] == idioma]
        cantidad_peliculas = len(df_peliculas_filtradas)
        return {'idioma': idioma, 'cantidad':cantidad_peliculas}
    else:
        return 'valor invalido'

In [99]:
peliculas_idioma('en')

{'idioma': 'en', 'cantidad': 32202}

### def peliculas_duracion(titulo)
- Se ingresa una pelicula y retorna la duracion y el anio de estreno.

In [100]:
#Convertir campo title a string minuscula
df_movies['title'] = df_movies['title'].astype(str).str.lower()

In [101]:
#Funcion ingreso de titulo y retorno de duracion y anio de la misma
def peliculas_duracion(titulo):
    titulo = titulo.lower()
    if titulo in df_movies['title'].values:
        df_pelicula = df_movies[df_movies['title'] == titulo]
        runtime = df_pelicula['runtime'].values[0]
        year = df_pelicula['year'].values[0]
        return {'titulo': titulo,'duracion':runtime,'anio': year}
    else:
        return 'valor invalido'

In [102]:
peliculas_duracion('happy burnout')

{'titulo': 'happy burnout', 'duracion': nan, 'anio': 2017}

### def franquicia(franquicia)
- Se ingresa la franquicia y retorna la cantidad de peliculas, ganancia total y ganancia promedio.

In [103]:
#Convertir campo collection a string minuscula
df_movies['collection'] = df_movies['collection'].str.lower()

In [104]:
#Funcion ingreso de franquicia y retorno de ganancia total y promedio
def franquicia(franquicia):
    franquicia = franquicia.lower()
    ganancia_total = 0
    if franquicia in df_movies['collection'].values:
        df_franquicia = df_movies[df_movies['collection'] == franquicia]
        ganancias = df_franquicia['revenue'].values[:]
        for ganancia in ganancias:
            ganancia_total += ganancia
        return {'franquicia': franquicia,'ganancia total':ganancia_total,'ganancia promedio': ganancia_total / len(ganancias)}
    else:
        return 'valor invalido'

In [105]:
franquicia('toy story collection')

{'franquicia': 'toy story collection',
 'ganancia total': 1937890605.0,
 'ganancia promedio': 645963535.0}

### def peliculas_pais(pais)
- Se ingresa un pais y retorna la cantidad de peliculas por pais.

In [106]:
#Convertir campo country a string minusculas
df_movies['country'] = df_movies['country'].str.lower()

In [107]:
#Funcion ingreso de pais y retorno de cantidad de peliculas por pais
def peliculas_pais(pais):
    pais = pais.lower()
    if pais in df_movies['country'].values:
        df_pais = df_movies[df_movies['country'] == pais]
        return {'pais': pais,'cantidad de peliculas':len(df_pais)}
    else:
        return 'valor invalido'

In [108]:
peliculas_pais('United StATes oF america')

{'pais': 'united states of america', 'cantidad de peliculas': 17846}

### def productoras_exitosas(productora)
- Se ingresa la productora y retorna el revenue total y la cantidad de peliculas que realizo

In [109]:
#Convertir campo companies a string minusculas
df_movies['companies'] = df_movies['companies'].str.lower()

In [110]:
#Funcion ingreso de productora y retorno de cantidad de peliculas y ganancia total de la productora
def productoras_exitosas(productora):
    productora = productora.lower()
    ganancia_total = 0
    df_productoras = df_movies[df_movies['companies'].str.contains(productora)]
    if len(df_productoras) != 0:
        ganancias = df_productoras['revenue'].values[:]
        for ganancia in ganancias:
            ganancia_total += ganancia
        return {'productora':productora,'cantidad de peliculas':len(df_productoras),'ganancia total':ganancia_total}
    else:
        return 'valor invalido'

In [111]:
productoras_exitosas('Tristar Pictures')

{'productora': 'tristar pictures',
 'cantidad de peliculas': 197,
 'ganancia total': 8022366764.0}

### def get_director(director)
- Se ingresa en director y retorna el nombre de la pelicula, la fecha de lanzamiento, retorno individual, costo y ganancia.

In [112]:
#Funcion para extraer los valores de la clave 'name'
def extract_values(value):
    if isinstance(value, str):
        values = re.findall(r"'Director', 'name': '(.*?)'", value)
        return ', '.join(values)
    else:
        return ''

#Desanidar campo crew del dataset credit    
df_credits['director'] = df_credits['crew'].apply(extract_values)

#Eliminar campos innecesarios
df_credits = df_credits.drop(columns=['cast', 'crew'])


In [113]:
#Convertir los valores id a int para realizar un merge
df_movies['id'] = pd.to_numeric(df_movies['id'], errors='coerce').fillna(0)
df_movies['id'] = df_movies['id'].astype('int64')

#Merge entre datasets movies y credit 
df_movies = pd.merge(df_movies, df_credits, on='id', how='inner')

In [114]:
#Convertir campo country a string minusculas
df_movies['director'] = df_movies['director'].str.lower()

In [115]:
#Funcion para agregar un prefijo a cada array 
def agregar_prefijo(array, prefijo):
    array_nuevo = []
    for elemento in array:
        nuevo_elemento = f'{prefijo}:{elemento}'
        array_nuevo.append(nuevo_elemento)
    return np.array(array_nuevo)

# Funcion ingreso de director y retorno de titulo, fecha, retorno de inversion, costo y ganancia de cada pelicula
def get_director(director):
    director = director.lower()
    df_peliculas = df_movies[df_movies['director'] == director]
    if len(df_peliculas) != 0:
        titulo = df_peliculas['title'].values[:]
        titulo = agregar_prefijo(titulo, 'titulo')
        fecha64 = df_peliculas['release_date'].values[:]
        fecha = np.datetime_as_string(fecha64, unit='D')
        fecha = agregar_prefijo(fecha, 'fecha de lanzamiento')
        retorno = df_peliculas['return'].values[:]
        retorno = agregar_prefijo(retorno, 'retorno')
        costo = df_peliculas['budget'].values[:]
        costo = agregar_prefijo(costo, 'costo:')  
        ganancia = df_peliculas['revenue'].values[:]
        ganancia = agregar_prefijo(ganancia, 'ganancia')   

        lista = list(zip(titulo, fecha, retorno, costo, ganancia))  
        return f'director:{director}', lista

In [116]:
get_director('charles shyer')

('director:charles shyer',
 [('titulo:father of the bride part ii',
   'fecha de lanzamiento:1995-02-10',
   'retorno:0.0',
   'costo::0.0',
   'ganancia:76578911.0'),
  ('titulo:i love trouble',
   'fecha de lanzamiento:1994-06-29',
   'retorno:1.3766059333333334',
   'costo::45000000.0',
   'ganancia:61947267.0'),
  ('titulo:baby boom',
   'fecha de lanzamiento:1987-09-17',
   'retorno:0.0',
   'costo::0.0',
   'ganancia:0.0'),
  ('titulo:the affair of the necklace',
   'fecha de lanzamiento:2001-11-20',
   'retorno:0.0',
   'costo::0.0',
   'ganancia:0.0'),
  ('titulo:father of the bride',
   'fecha de lanzamiento:1991-12-20',
   'retorno:0.0',
   'costo::0.0',
   'ganancia:89325780.0'),
  ('titulo:alfie',
   'fecha de lanzamiento:2004-10-22',
   'retorno:0.22326565',
   'costo::60000000.0',
   'ganancia:13395939.0'),
  ('titulo:irreconcilable differences',
   'fecha de lanzamiento:1984-09-28',
   'retorno:0.0',
   'costo::0.0',
   'ganancia:12384120.0')])

# Creacion de dataframe con los campos modificados  

In [117]:
if not os.path.isfile('movies_etl.csv'):
    df_movies.to_csv('movies_etl.csv', index=False)
    print('Archivo movies_etl.csv creado')
else:
    print('Archivo existente')

Archivo existente


In [118]:
#Codigo alternativos
#df.isna().sum()

#count_type = df['column'].apply(type).value_counts()
#for tipo, cantidad in count_type.items():
#    print(tipo, cantidad)

#data_type = type(df.iloc[row, column])
#print(data_type)